In [11]:
class Node(
    var key: Int, var left: Node? = null, var right: Node? = null
) {
    fun find(value: Int): Node? = when {
        this.key > value -> left?.find(value)
        this.key < value -> right?.find(value)
        else -> this
    }

    fun insert(value: Int) {
        if (value > this.key) {
            if (this.right == null) {
                this.right = Node(value)
            } else {
                this.right?.insert(value)
            }
        } else if (value < this.key) {
            if (this.left == null) {
                this.left = Node(value)
            } else {
                this.left?.insert(value)
            }
        }
    }

    fun delete(value: Int) {
        when {
            value > key -> scan(value, this.right, this)
            value < key -> scan(value, this.left, this)
            else -> removeNode(this, null)
        }
    }

    private fun scan(value: Int, node: Node?, parent: Node?) {
        if (node == null) {
            println("value $value seems not present in the tree.")
            return
        }
        when {
            value > node.key -> scan(value, node.right, node)
            value < node.key -> scan(value, node.left, node)
            value == node.key -> removeNode(node, parent)
        }
    }

    private fun removeNode(node: Node, parent: Node?) {
        node.left?.let { leftChild ->
            run {
                node.right?.let {
                    removeTwoChildNode(node)
                } ?: removeSingleChildNode(node, leftChild)
            }
        } ?: run {
            node.right?.let { rightChild -> removeSingleChildNode(node, rightChild) } ?: removeNoChildNode(node, parent)
        }
    }

    private fun removeNoChildNode(node: Node, parent: Node?) {
        parent?.let { p ->
            if (node == p.left) {
                p.left = null
            } else if (node == p.right) {
                p.right = null
            }
        } ?: error("Can not remove the root node without child nodes")
    }

    private fun removeTwoChildNode(node: Node) {
        val leftChild = node.left!!
        leftChild.right?.let {
            val maxParent = findParentOfMaxChild(leftChild)
            maxParent.right?.let {
                node.key = it.key
                maxParent.right = null
            } ?: error("Node with max child must have the right child!")
        } ?: run {
            node.key = leftChild.key
            node.left = leftChild.left
        }
    }

    private fun findParentOfMaxChild(n: Node): Node {
        return n.right?.let { r -> r.right?.let { findParentOfMaxChild(r) } ?: n }
            ?: throw IllegalArgumentException("Right child must be non-null")
    }

    private fun removeSingleChildNode(parent: Node, child: Node) {
        parent.key = child.key
        parent.left = child.left
        parent.right = child.right
    }

    fun visit(): IntArray {
        val a = left?.visit() ?: intArrayOf()
        val b = right?.visit() ?: intArrayOf()
        return a + intArrayOf(key) + b
    }
}

In [12]:
class BTreePrinter {
    fun printNode(root: Node?) {
        val maxLevel = maxLevel(root)
        printNodeInternal(listOf(root), 1, maxLevel)
    }

    private fun printNodeInternal(
        nodes: List<Node?>, level: Int, maxLevel: Int
    ) {
        if (nodes.isEmpty() || isAllElementsNull(nodes)) return
        val floor = maxLevel - level
        val edgeLines = 2.0.pow((floor - 1).coerceAtLeast(0).toDouble()).toInt()
        val firstSpaces = 2.0.pow(floor.toDouble()).toInt() - 1
        val betweenSpaces = 2.0.pow((floor + 1).toDouble()).toInt() - 1
        printWhitespaces(firstSpaces)
        val newNodes: MutableList<Node?> = ArrayList()
        for (node in nodes) {
            if (node != null) {
                print(node.key)
                newNodes.add(node.left)
                newNodes.add(node.right)
            } else {
                newNodes.add(null)
                newNodes.add(null)
                print(" ")
            }
            printWhitespaces(betweenSpaces)
        }
        println("")
        for (i in 1..edgeLines) {
            for (j in nodes.indices) {
                printWhitespaces(firstSpaces - i)
                if (nodes[j] == null) {
                    printWhitespaces(edgeLines + edgeLines + i + 1)
                    continue
                }
                if (nodes[j]!!.left != null) print("/") else printWhitespaces(1)
                printWhitespaces(i + i - 1)
                if (nodes[j]!!.right != null) print("\\") else printWhitespaces(1)
                printWhitespaces(edgeLines + edgeLines - i)
            }
            println("")
        }
        printNodeInternal(newNodes, level + 1, maxLevel)
    }

    private fun printWhitespaces(count: Int) {
        for (i in 0 until count) print(" ")
    }

    private fun maxLevel(node: Node?): Int {
        return if (node == null) 0 else maxLevel(node.left).coerceAtLeast(maxLevel(node.right)) + 1
    }

    private fun <T> isAllElementsNull(list: List<T?>): Boolean {
        for (`object` in list) {
            if (`object` != null) return false
        }
        return true
    }
}

In [13]:
val tree = Node(4)
val keys = arrayOf(1, 2, 3, 4, 5, 6, 7)

for (key in keys) {
    tree.insert(key)
}

val node = tree.find(4)!!
println("Node with value ${node.key} [left = ${node.left?.key}, right = ${node.right?.key}]")
println("Delete node with key = 3")

//node.delete(3)

print("Tree content after the node elimination: ")
println(tree.visit().joinToString { it.toString() })

Node with value 4 [left = 1, right = 5]
Delete node with key = 3
Tree content after the node elimination: 1, 2, 3, 4, 5, 6, 7


In [14]:
BTreePrinter().printNode(tree)

       4               
      / \       
     /   \      
    /     \     
   /       \    
   1       5       
    \       \   
     \       \  
     2       6   
      \       \ 
      3       7 
                                
